In [1]:
from urllib.request import Request, urlopen as Ureq
import urllib.request
import requests
import PyPDF2
from analysis import tokenizer, twitter_sent_analysis
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from bs4 import BeautifulSoup as soup
import pandas as pd
import os
from collections import Counter
import time
import sys

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\cscat\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
class WhitepapersDB():

    def __init__(self):
        self.base_url = 'https://whitepaperdatabase.com/?s='


    def base_page(self, term):
        new_url = self.base_url + term
        print(new_url)
        uClient = Ureq(new_url)
        raw_content = uClient.read()
        uClient.close()
        page_soup = soup(raw_content)
        return page_soup

    def get_paper_url(self, page_soup):
        containers = page_soup.findAll("a")
        url = containers[8]['href']
        return url

    def get_pdf_link(self, paper_url):
        uClient = Ureq(paper_url)
        raw_content = uClient.read()
        uClient.close()
        page_soup = soup(raw_content)
        pdf_link = page_soup.findAll("a", {"class":"pdfemb-viewer"})
        return pdf_link[0]['href']

    def get_pdf(self, ticker, pdf_link):
        filename = '../../data/whitepapers/' + ticker + '_whitepaper.pdf'
        urllib.request.urlretrieve(pdf_link, filename)

    def read_pdf(self, ticker):
        corpus = ''
        filename = '../../data/whitepapers/' + ticker + '_whitepaper.pdf'
        pdf_obj = open(filename, 'rb')
        pdfReader = PyPDF2.PdfFileReader(pdf_obj)
        pages = pdfReader.numPages
        for i in range(pages):
            raw_text = pdfReader.getPage(i)
            corpus = corpus + raw_text.extractText()
        return corpus
    
    def preprocess_sent(text):
        tokens = tokenizer(text)
        corpus = [corpus + ' ' + word for word in tokens]
        return corpus[0]

    def check_sent(self, corpus):
        analyzer = SentimentIntensityAnalyzer()
        sentiment = analyzer.polarity_scores(corpus)
        return sentiment